In [1]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.11.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  404k    0  404k    0     0   859k      0 --:--:-- --:--:-- --:--:-- 4898k
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/pytorch3d.git (to revision stable) to /tmp/pip-req-build-jy92dj8l
  Running command git clone -q https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-jy92dj8l
  Running command git checkout -q 276c9a8acbfa715f5802f26ec9f4141bde26ecb4
     |████████████████████████████████| 50 kB 3.7 MB/s 
     |████████████████████████████████| 42 kB 554 kB/s 
  Created wheel for pytorch3d: filename=pytorch3d-0.7.0-cp37-cp37m-linux_x86_64.whl size=24897096 sha256=a0a2b352220559f8290492ad1414027d97f90bc24bc50a4397b9fa757fae831c
  Store

In [2]:
!git clone https://github.com/learning3d/assignment1.git


Cloning into 'assignment1'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 46 (delta 9), reused 40 (delta 7), pack-reused 0
Unpacking objects: 100% (46/46), done.


In [3]:
%cd assignment1
!pip install -r requirements.txt

/content/assignment1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 265 kB 10.8 MB/s 


In [4]:
import argparse
import pickle

import matplotlib.pyplot as plt
import mcubes
import numpy as np
import pytorch3d
import torch


def load_rgbd_data(path="data/rgbd_data.pkl"):
    with open(path, "rb") as f:
        data = pickle.load(f)
    return data

In [5]:
data=load_rgbd_data()
data

{'rgb1': array([[[0.81960785, 0.8039216 , 0.80784315],
         [0.81960785, 0.8039216 , 0.80784315],
         [0.8187211 , 0.8030349 , 0.8069564 ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],
 
        [[0.81960785, 0.8039216 , 0.80784315],
         [0.81960785, 0.8039215 , 0.80784315],
         [0.8182827 , 0.80259645, 0.806518  ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],
 
        [[0.81828433, 0.802598  , 0.8065196 ],
         [0.8196078 , 0.8039216 , 0.80784315],
         [0.8174043 , 0.80171794, 0.80563956],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],
 
        ...,
 
        [[0.6610104 , 0.20610838, 0.2806182 ],
         [0.6588363 , 0.20393431,

In [18]:
data['cameras1'].in_ndc()

True

In [6]:
data['rgb1'].shape

(800, 800, 3)

In [7]:
def unproject_depth_image(image, mask, depth, camera):
    """
    Unprojects a depth image into a 3D point cloud.

    Args:
        image (torch.Tensor): A square image to unproject (S, S, 3).
        mask (torch.Tensor): A binary mask for the image (S, S).
        depth (torch.Tensor): The depth map of the image (S, S).
        camera: The Pytorch3D camera to render the image.
    
    Returns:
        points (torch.Tensor): The 3D points of the unprojected image (N, 3).
        rgba (torch.Tensor): The rgba color values corresponding to the unprojected
            points (N, 4).
    """
    device = camera.device
    assert image.shape[0] == image.shape[1], "Image must be square."
    image_shape = image.shape[0]
    ndc_pixel_coordinates = torch.linspace(1, -1, image_shape)
    Y, X = torch.meshgrid(ndc_pixel_coordinates, ndc_pixel_coordinates)
    xy_depth = torch.dstack([X, Y, depth])
    points = camera.unproject_points(
        xy_depth.to(device), in_ndc=False, from_ndc=False, world_coordinates=True,
    )
    points = points[mask > 0.5]
    rgb = image[mask > 0.5]
    rgb = rgb.to(device)

    # For some reason, the Pytorch3D compositor does not apply a background color
    # unless the pointcloud is RGBA.
    alpha = torch.ones_like(rgb)[..., :1]
    rgb = torch.cat([rgb, alpha], dim=1)

    return points, rgb

In [8]:
pointsCloud1=unproject_depth_image(torch.from_numpy(data['rgb1']),torch.from_numpy(data['mask1']),torch.from_numpy(data['depth1']),data['cameras1'])
pointsCloud2=unproject_depth_image(torch.from_numpy(data['rgb2']),torch.from_numpy(data['mask2']),torch.from_numpy(data['depth2']),data['cameras2'])

/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [9]:
from pytorch3d.structures import Pointclouds
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")
verts_1 = torch.Tensor(pointsCloud1[0]).to(device)
verts_2 = torch.Tensor(pointsCloud2[0]).to(device)
        
rgb_1 = torch.Tensor(pointsCloud1[1]).to(device)
rgb_2 = torch.Tensor(pointsCloud2[1]).to(device)

points_cloud_1 = Pointclouds(points=[verts_1], features=[rgb_1])
points_cloud_2 = Pointclouds(points=[verts_2], features=[rgb_2])
points_cloud_3 = Pointclouds(points=[verts_1,verts_2], features=[rgb_1,rgb_2])

In [11]:
plot_scene({
    "Pointcloud": {
        "Pytorch3D": points_cloud_1
    }
})

In [12]:
plot_scene({
    "Pointcloud": {
        "Pytorch3D": points_cloud_2
    }
})

In [13]:
plot_scene({
    "Pointcloud": {
        "Pytorch3D": points_cloud_3
    }
})

Output hidden; open in https://colab.research.google.com to view.